# Chapter 3: Classification

Chapter ini membahas secara mendalam salah satu pilar utama dalam supervised learning, yaitu **classification**. Classification merupakan proses pembelajaran mesin yang bertujuan untuk memprediksi **kelas diskrit** berdasarkan sekumpulan fitur input.

Dalam praktiknya, classification digunakan secara luas dalam berbagai bidang, seperti pengenalan pola, pengolahan citra, keamanan siber, sistem rekomendasi, hingga bidang kesehatan. Oleh karena itu, pemahaman yang kuat terhadap konsep classification dan evaluasinya merupakan fondasi penting dalam Machine Learning.

Pada chapter ini, seluruh konsep classification akan dijelaskan secara sistematis menggunakan dataset **MNIST**, sebagaimana dibahas dalam buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow* karya Aurélien Géron.

## 1. What Is Classification?

Classification adalah bentuk supervised learning di mana model mempelajari hubungan antara data input (fitur) dan output berupa **label kategorikal**. Label ini bersifat diskrit dan telah ditentukan sebelumnya.

Tidak seperti regresi yang menghasilkan nilai numerik kontinu, classification menghasilkan keputusan diskrit, misalnya:
- spam atau bukan spam,
- fraud atau transaksi normal,
- digit 0 sampai 9,
- positif atau negatif.

Tujuan utama classification bukan hanya menghasilkan prediksi yang benar, tetapi juga memahami **konsekuensi dari kesalahan prediksi**, terutama dalam konteks dunia nyata seperti diagnosis medis atau sistem keamanan.

## 2. The MNIST Dataset

MNIST adalah dataset benchmark klasik yang digunakan untuk tugas classification. Dataset ini terdiri dari **70.000 gambar digit tulisan tangan**, yang masing-masing merepresentasikan angka dari 0 hingga 9.

Setiap gambar memiliki ukuran 28 × 28 piksel dan direpresentasikan sebagai vektor berdimensi 784. Nilai piksel berada dalam rentang 0–255, di mana nilai yang lebih tinggi merepresentasikan warna yang lebih terang.

Meskipun tampak sederhana, MNIST mengandung variasi tulisan tangan yang cukup kompleks, menjadikannya dataset yang ideal untuk mempelajari kesalahan classification dan evaluasi model.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']

Label pada dataset MNIST disimpan dalam bentuk string. Untuk keperluan pemodelan, label ini perlu dikonversi ke tipe numerik.

In [ ]:
import numpy as np
y = y.astype(np.uint8)

## 3. Visualizing a Digit

Sebelum membangun model, sangat penting untuk memahami bentuk dan struktur data yang digunakan. Visualisasi data membantu memverifikasi integritas data serta memberikan intuisi awal terhadap kompleksitas permasalahan.

Vektor berdimensi 784 dapat dikonversi kembali menjadi citra 28 × 28 untuk divisualisasikan.

In [ ]:
import matplotlib.pyplot as plt

some_digit = X.iloc[0]
some_digit_image = some_digit.values.reshape(28, 28)

plt.imshow(some_digit_image, cmap='binary')
plt.axis('off')
plt.show()

Visualisasi di atas menunjukkan salah satu contoh digit tulisan tangan. Variasi bentuk dan gaya penulisan inilah yang membuat tugas classification menjadi menantang.

## 4. Train–Test Split

Untuk memastikan evaluasi model yang objektif, dataset harus dibagi menjadi **training set** dan **test set**.

Training set digunakan untuk melatih model, sedangkan test set digunakan untuk mengevaluasi performa model pada data yang belum pernah dilihat sebelumnya.

Pada dataset MNIST, pembagian standar adalah 60.000 data training dan 10.000 data testing.

In [ ]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]